# IHDP dataset: Dragonnet vs Meta Learners

In [2]:
import os, sys

In [3]:
sys.path.append('../causalml/inference/nn')

In [4]:
%load_ext autoreload
%autoreload 2

In [19]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
import statsmodels.api as sm
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
import warnings

from causalml.inference.meta import LRSRegressor
from causalml.inference.meta import XGBTRegressor, MLPTRegressor
from causalml.inference.meta import BaseXRegressor, BaseRRegressor, BaseSRegressor, BaseTRegressor
from causalml.match import NearestNeighborMatch, MatchOptimizer, create_table_one
from causalml.propensity import ElasticNetPropensityModel
from causalml.dataset import *
from causalml.metrics import *

from dragonnet import DragonNet

warnings.filterwarnings('ignore')
plt.style.use('fivethirtyeight')

%matplotlib inline

In [ ]:
import causalml
print(causalml.__version__)

# Part A: IHDP dataset

In [6]:
ihdp_dir = '/Users/mike.yung/Documents/repos/dragonnet/dat/ihdp/csv'

In [7]:
sorted(os.listdir(ihdp_dir))

['ihdp_npci_1.csv',
 'ihdp_npci_10.csv',
 'ihdp_npci_11.csv',
 'ihdp_npci_12.csv',
 'ihdp_npci_13.csv',
 'ihdp_npci_14.csv',
 'ihdp_npci_15.csv',
 'ihdp_npci_16.csv',
 'ihdp_npci_17.csv',
 'ihdp_npci_18.csv',
 'ihdp_npci_19.csv',
 'ihdp_npci_2.csv',
 'ihdp_npci_20.csv',
 'ihdp_npci_21.csv',
 'ihdp_npci_22.csv',
 'ihdp_npci_23.csv',
 'ihdp_npci_24.csv',
 'ihdp_npci_25.csv',
 'ihdp_npci_26.csv',
 'ihdp_npci_27.csv',
 'ihdp_npci_28.csv',
 'ihdp_npci_29.csv',
 'ihdp_npci_3.csv',
 'ihdp_npci_30.csv',
 'ihdp_npci_31.csv',
 'ihdp_npci_32.csv',
 'ihdp_npci_33.csv',
 'ihdp_npci_34.csv',
 'ihdp_npci_35.csv',
 'ihdp_npci_36.csv',
 'ihdp_npci_37.csv',
 'ihdp_npci_38.csv',
 'ihdp_npci_39.csv',
 'ihdp_npci_4.csv',
 'ihdp_npci_40.csv',
 'ihdp_npci_41.csv',
 'ihdp_npci_42.csv',
 'ihdp_npci_43.csv',
 'ihdp_npci_44.csv',
 'ihdp_npci_45.csv',
 'ihdp_npci_46.csv',
 'ihdp_npci_47.csv',
 'ihdp_npci_48.csv',
 'ihdp_npci_49.csv',
 'ihdp_npci_5.csv',
 'ihdp_npci_50.csv',
 'ihdp_npci_6.csv',
 'ihdp_npci_7.csv',
 'ihdp_npci_8.csv',
 'ihdp_npci_9.csv']

In [10]:
df = pd.read_csv(f'{ihdp_dir}/ihdp_npci_1.csv', header=None)
cols =  ["treatment", "y_factual", "y_cfactual", "mu0", "mu1"] + [f'x{i}' for i in range(1,26)]
df.columns = cols

In [11]:
df.head()

,treatment,y_factual,y_cfactual,mu0,mu1,x1,x2,x3,x4,x5,...,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25
0,1,5.599916,4.318780,3.268256,6.854457,-0.528603,-0.343455,1.128554,0.161703,-0.316603,...,1,1,1,1,0,0,0,0,0,0
1,0,6.875856,7.856495,6.636059,7.562718,-1.736945,-1.802002,0.383828,2.244320,-0.629189,...,1,1,1,1,0,0,0,0,0,0
2,0,2.996273,6.633952,1.570536,6.121617,-0.807451,-0.202946,-0.360898,-0.879606,0.808706,...,1,0,1,1,0,0,0,0,0,0
3,0,1.366206,5.697239,1.244738,5.889125,0.390083,0.596582,-1.850350,-0.879606,-0.004017,...,1,0,1,1,0,0,0,0,0,0
4,0,1.963538,6.202582,1.685048,6.191994,-1.045229,-0.602710,0.011465,0.161703,0.683672,...,1,1,1,1,0,0,0,0,0,0


In [12]:
pd.Series(df['treatment']).value_counts(normalize=True)

0    0.813922
1    0.186078
Name: treatment, dtype: float64

In [14]:
X = df.loc[:,'x1':]
treatment = df['treatment']
y = df['y_factual']
tau = df.apply(lambda d: d['y_factual'] - d['y_cfactual']
               if d['treatment']==1
               else d['y_cfactual'] - d['y_factual'], axis=1)

In [16]:
p_model = LogisticRegressionCV(penalty='elasticnet', solver='saga', l1_ratios=np.linspace(0,1,10),
                               cv=StratifiedKFold(n_splits=2, shuffle=False))
p_model.fit(X, treatment)
p = p_model.predict_proba(X)[:, 1]

In [17]:
s_learner = BaseSRegressor(LGBMRegressor())
s_ate = s_learner.estimate_ate(X, treatment, y)[0]
s_ite = s_learner.fit_predict(X, treatment, y)

t_learner = BaseTRegressor(LGBMRegressor())
t_ate = t_learner.estimate_ate(X, treatment, y)[0][0]
t_ite = t_learner.fit_predict(X, treatment, y)

x_learner = BaseXRegressor(LGBMRegressor())
x_ate = x_learner.estimate_ate(X, p, treatment, y)[0][0]
x_ite = x_learner.fit_predict(X, p, treatment, y)

r_learner = BaseRRegressor(LGBMRegressor())
r_ate = r_learner.estimate_ate(X, p, treatment, y)[0][0]
r_ite = r_learner.fit_predict(X, p, treatment, y)

In [40]:
y.shape

(747,)

In [41]:
X.shape

(747, 25)

In [42]:
dragon = DragonNet(neurons_per_layer=200, targeted_reg=True)
dragon_ite = dragon.fit_predict(X, treatment, y, return_components=False)
dragon_ate = dragon_ite.mean()

Train on 597 samples, validate on 150 samples
Epoch 1/30
597/597 [==============================] - 1s 1ms/step - loss: 789.8568 - regression_loss: 350.3596 - binary_classification_loss: 36.9470 - treatment_accuracy: 0.7638 - track_epsilon: 0.0564 - val_loss: 432.2341 - val_regression_loss: 169.8363 - val_binary_classification_loss: 32.7280 - val_treatment_accuracy: 0.7244 - val_track_epsilon: 0.0569
Epoch 2/30
597/597 [==============================] - 0s 97us/step - loss: 288.7052 - regression_loss: 124.5967 - binary_classification_loss: 27.4776 - treatment_accuracy: 0.8561 - track_epsilon: 0.0576 - val_loss: 309.4809 - val_regression_loss: 105.3295 - val_binary_classification_loss: 34.3288 - val_treatment_accuracy: 0.7244 - val_track_epsilon: 0.0590
Epoch 3/30
597/597 [==============================] - 0s 101us/step - loss: 238.4009 - regression_loss: 95.1742 - binary_classification_loss: 25.7477 - treatment_accuracy: 0.8561 - track_epsilon: 0.0580 - val_loss: 234.0874 - val_regress

Epoch 11/300
597/597 [==============================] - 0s 74us/step - loss: 140.3434 - regression_loss: 52.9613 - binary_classification_loss: 23.2865 - treatment_accuracy: 0.8593 - track_epsilon: 0.0079 - val_loss: 191.6038 - val_regression_loss: 61.2092 - val_binary_classification_loss: 35.3782 - val_treatment_accuracy: 0.7244 - val_track_epsilon: 0.0084
Epoch 12/300
597/597 [==============================] - 0s 95us/step - loss: 139.0111 - regression_loss: 52.7522 - binary_classification_loss: 23.2591 - treatment_accuracy: 0.8401 - track_epsilon: 0.0082 - val_loss: 187.9949 - val_regression_loss: 59.8345 - val_binary_classification_loss: 35.4621 - val_treatment_accuracy: 0.7244 - val_track_epsilon: 0.0055
Epoch 13/300
597/597 [==============================] - 0s 86us/step - loss: 138.3175 - regression_loss: 52.8982 - binary_classification_loss: 23.2560 - treatment_accuracy: 0.8433 - track_epsilon: 0.0047 - val_loss: 186.3428 - val_regression_loss: 59.3087 - val_binary_classificatio

Epoch 34/300
597/597 [==============================] - 0s 83us/step - loss: 137.0716 - regression_loss: 51.5655 - binary_classification_loss: 23.0700 - treatment_accuracy: 0.8497 - track_epsilon: 0.0055 - val_loss: 191.6972 - val_regression_loss: 61.5732 - val_binary_classification_loss: 35.1340 - val_treatment_accuracy: 0.7244 - val_track_epsilon: 0.0047
Epoch 35/300
597/597 [==============================] - 0s 72us/step - loss: 137.6223 - regression_loss: 51.8798 - binary_classification_loss: 23.0633 - treatment_accuracy: 0.8497 - track_epsilon: 0.0043 - val_loss: 192.0596 - val_regression_loss: 61.7552 - val_binary_classification_loss: 35.1869 - val_treatment_accuracy: 0.7244 - val_track_epsilon: 0.0044

Epoch 00035: ReduceLROnPlateau reducing learning rate to 2.499999936844688e-06.
Epoch 36/300
597/597 [==============================] - 0s 69us/step - loss: 136.9475 - regression_loss: 51.4183 - binary_classification_loss: 23.0654 - treatment_accuracy: 0.8593 - track_epsilon: 0.00

597/597 [==============================] - 0s 92us/step - loss: 134.9833 - regression_loss: 51.1673 - binary_classification_loss: 23.0352 - treatment_accuracy: 0.8529 - track_epsilon: 0.0050 - val_loss: 191.3936 - val_regression_loss: 61.4206 - val_binary_classification_loss: 35.0937 - val_treatment_accuracy: 0.7244 - val_track_epsilon: 0.0049


In [391]:
mae = lambda true,pred: np.mean(np.abs(true - pred))

In [53]:
df_preds = pd.DataFrame([s_ite.ravel(),
                          t_ite.ravel(),
                          x_ite.ravel(),
                          r_ite.ravel(),
                          dragon_ite.ravel(),
                          tau.ravel(),
                          treatment.ravel(),
                          y.ravel()],
                       index=['S','T','X','R','dragonnet','tau','w','y']).T

df_cumgain = get_cumgain(df_preds)

In [54]:
df_result = pd.DataFrame([s_ate, t_ate, x_ate, r_ate, dragon_ate, tau.mean()],
                     index=['S','T','X','R','dragonnet','actual'], columns=['ATE'])
df_result['MAE'] = [mae(t,p) for t,p in zip([s_ite, t_ite, x_ite, r_ite, dragon_ite],
                                         [tau.values.reshape(-1,1)]*5 )
                ] + [None]
df_result['AUUC'] = auuc_score(df_preds)

In [55]:
df_result

,ATE,MAE,AUUC
S,3.931390,1.031916,0.562590
T,3.968422,1.007734,0.566385
X,4.004151,1.136789,0.547443
R,3.137150,1.949508,0.549099
dragonnet,3.983296,1.124654,0.538762
actual,4.029661,NaN,NaN


### Loop through all files

In [57]:
df_result_all = pd.DataFrame()

for filename in sorted(os.listdir(ihdp_dir))[:10]:
    df = pd.read_csv(f'{ihdp_dir}/{filename}', header=None)
    col =  ["treatment", "y_factual", "y_cfactual", "mu0", "mu1" ]

    for i in range(1,26):
        col.append("x"+str(i))
    df.columns = col

    X = df.loc[:,'x1':]
    treatment = df['treatment']
    y = df['y_factual']
    tau = df.apply(lambda d: d['y_factual'] - d['y_cfactual']
                   if d['treatment']==1
                   else d['y_cfactual'] - d['y_factual'], axis=1)

    p_model = LogisticRegressionCV(penalty='elasticnet', solver='saga', l1_ratios=np.linspace(0,1,4),
                                   cv=StratifiedKFold(n_splits=3, shuffle=False))
    p_model.fit(X, treatment)
    p = p_model.predict_proba(X)[:, 1]

    s_learner = BaseSRegressor(XGBRegressor())
    s_ate = s_learner.estimate_ate(X, treatment, y)[0]
    s_ite = s_learner.fit_predict(X, treatment, y)

    t_learner = BaseTRegressor(XGBRegressor())
    t_ate = t_learner.estimate_ate(X, treatment, y)[0][0]
    t_ite = t_learner.fit_predict(X, treatment, y)

    x_learner = BaseXRegressor(XGBRegressor())
    x_ate = x_learner.estimate_ate(X, p, treatment, y)[0][0]
    x_ite = x_learner.fit_predict(X, p, treatment, y)

    r_learner = BaseRRegressor(XGBRegressor())
    r_ate = r_learner.estimate_ate(X, p, treatment, y)[0][0]
    r_ite = r_learner.fit_predict(X, p, treatment, y)

    dragon = DragonNet(neurons_per_layer=200, targeted_reg=True, verbose=False)
    dragon_ite = dragon.fit_predict(X, treatment, y, return_components=False)
    dragon_ate = dragon_ite.mean()

    mae = lambda true,pred: np.mean(np.abs(true - pred))

    df_preds = pd.DataFrame([s_ite.ravel(),
                              t_ite.ravel(),
                              x_ite.ravel(),
                              r_ite.ravel(),
                              dragon_ite.ravel(),
                              tau.ravel(),
                              treatment.ravel(),
                              y.ravel()],
                           index=['S','T','X','R','dragonnet','tau','w','y']).T

    df_cumgain = get_cumgain(df_preds)

    df_result = pd.DataFrame([s_ate, t_ate, x_ate, r_ate, dragon_ate, tau.mean()],
                         index=['S','T','X','R','dragonnet','actual'], columns=['ATE'])
    df_result['MAE'] = [mae(t,p) for t,p in zip([s_ite, t_ite, x_ite, r_ite, dragon_ite],
                                             [tau.values.reshape(-1,1)]*5 )
                    ] + [None]
    df_result['AUUC'] = auuc_score(df_preds)
    df_result.reset_index(inplace=True)
    df_result['file'] = filename
    df_result_all = pd.concat((df_result_all, df_result))
    
    print(f'Done with {filename}')

df_result_all.rename({'index':'model'}, axis=1, inplace=True)

# manual adjustment
df_result_all.loc[df_result_all.apply(lambda d: d['model']=='dragonnet' and np.isnan(d['MAE']), axis=1),
                 'AUUC'] = np.nan

Done with ihdp_npci_1.csv
Batch 5: Invalid loss, terminating training
Done with ihdp_npci_10.csv
Done with ihdp_npci_11.csv
Done with ihdp_npci_12.csv
Batch 3: Invalid loss, terminating training
Done with ihdp_npci_13.csv
Done with ihdp_npci_14.csv
Done with ihdp_npci_15.csv
Done with ihdp_npci_16.csv
Done with ihdp_npci_17.csv
Done with ihdp_npci_18.csv


In [58]:
df_result_all.groupby(['file','model']).mean()

ATE        MAE      AUUC
file             model                                    
ihdp_npci_1.csv  R           3.392378   1.379784  0.539915
                 S           3.887754   1.108337  0.545272
                 T           3.949531   1.087281  0.551643
                 X           4.010829   1.130968  0.543934
                 actual      4.029661        NaN       NaN
                 dragonnet   3.962618   1.125623  0.539087
ihdp_npci_10.csv R           3.857500   2.286114  0.994029
                 S           4.738152   2.358156  1.002430
                 T           4.696530   1.414504  1.009496
                 X           4.619070   2.179820  0.996405
                 actual      4.640317        NaN       NaN
                 dragonnet        NaN        NaN       NaN
ihdp_npci_11.csv R           3.494348   1.390417  0.618886
                 S           3.977760   1.307770  0.626742
                 T           4.003299   1.146260  0.632129
                 X           3.962137   1.245397  0.626406
                 actual      3.837335        NaN       NaN
                 dragonnet   3.839352   1.219462  0.626603
ihdp_npci_12.csv R           3.418269   1.819194  0.739296
                 S           4.269590   1.716805  0.754728
                 T           4.111572   1.202795  0.760183
                 X           4.134551   1.645744  0.746968
                 actual      4.195788        NaN       NaN
                 dragonnet   4.175613   1.202777  0.761143
ihdp_npci_13.csv R           8.581749  12.052369  1.170040
                 S          13.121904  14.643461  1.164333
                 T          12.888197   7.017251  1.211689
                 X          12.178790  11.042119  1.184904
                 actual     12.653138        NaN       NaN
                 dragonnet        NaN        NaN       NaN
ihdp_npci_14.csv R           3.347821   1.398600  0.544212
                 S           3.747818   1.145911  0.546522
                 T           3.818774   1.094771  0.558276
                 X           3.875319   1.182330  0.541974
                 actual      3.844052        NaN       NaN
                 dragonnet   3.834915   1.245932  0.537838
ihdp_npci_15.csv R           3.395916   1.449394  0.627455
                 S           4.030133   1.532327  0.617098
                 T           4.060164   1.180957  0.640933
                 X           4.018644   1.364685  0.628980
                 actual      3.929148        NaN       NaN
                 dragonnet   4.003075   1.243092  0.637877
ihdp_npci_16.csv R           3.911443   1.597002  0.548937
                 S           4.488440   1.254707  0.572632
                 T           4.698523   1.132001  0.580085
                 X           4.534673   1.238216  0.570628
                 actual      4.564828        NaN       NaN
                 dragonnet   4.449808   1.228915  0.571797
ihdp_npci_17.csv R           3.022923   1.752116  0.807123
                 S           3.690936   2.255814  0.795397
                 T           3.757145   1.259859  0.823976
                 X           3.756283   1.729303  0.808584
                 actual      3.679116        NaN       NaN
                 dragonnet   3.527126   1.289683  0.823617
ihdp_npci_18.csv R           3.011991   1.484006  0.715327
                 S           3.511694   1.533943  0.711287
                 T           3.635561   1.139114  0.725727
                 X           3.655067   1.364692  0.717772
                 actual      3.712564        NaN       NaN
                 dragonnet   3.421735   1.189114  0.723864

### Compare performance with result from simulation script

In [9]:
import pickle

In [80]:
file_idx = 42

with np.load(f'/Users/mike.yung/Documents/repos/dragonnet/result/ihdp/dragonnet/{file_idx}/baseline/0_replication_train.npz') as f:
    _pred = f['q_t1'] - f['q_t0']
    _idx = f['index']

with np.load(f'/Users/mike.yung/Documents/repos/dragonnet/result/ihdp/dragonnet/{file_idx}/simulation_outputs.npz') as f:
    _t = f['t']
    _y = f['y']
    _y_cf = f['y_cf']    

df_sim = pd.DataFrame({'y':_y.ravel(),
                       'y_cf': _y_cf.ravel(),
                       't': _t.ravel(),
})
df_sim['tau'] = df_sim.apply(lambda d: d['y'] - d['y_cf'] if d['t']==1 else d['y_cf'] - d['y'], axis=1)    

print(mae(df_sim.loc[_idx, 'tau'], _pred))
df_result_all[df_result_all['file']==f'ihdp_npci_{file_idx+1}.csv']

1.2460000174576988


,model,ATE,MAE,AUUC,file
0,S,4.417201,1.239931,0.583065,ihdp_npci_43.csv
1,T,4.618019,1.088651,0.594821,ihdp_npci_43.csv
2,X,4.415116,1.181548,0.586271,ihdp_npci_43.csv
3,R,3.898648,1.490834,0.572499,ihdp_npci_43.csv
4,dragonnet,4.394847,1.128743,0.591419,ihdp_npci_43.csv
5,actual,4.454654,NaN,NaN,ihdp_npci_43.csv
